In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated


Ulepszony pomiar - ustawienie w GA4 automatycznie zbiera pewne wydarzenia na stronie internetowej, a te są również zapisywane do eksportu BigQuery.

Zdarzenia te mogą mieć własne, unikalne metadane, dlatego warto przeprowadzić analizę eksploracyjną przed uruchomieniem zapytań.

Użyj PIVOT, aby przekonwertować wiersze na kolumny. Jest to przydatne, jeśli chcesz tworzyć czytelne, zagregowane raporty, w których dane są układane w kolumny, a nie w wiersze.


Count Scroll Interactions

In [ ]:
%%bigquery --project 

-- METHOD 1: Query all scroll events
select
    (select value.string_value from unnest(event_params) where event_name = 'scroll' and key = 'page_location') as scroll_page,
    countif(event_name = 'scroll') as scrolls
from 
    `ga.events_20210601`
group by 
    scroll_page
order by 
    scrolls desc

In [ ]:
%%bigquery --project 

-- METHOD 2: Query only scrolls that reach a certain threshold
select
    case 
        when (select value.int_value from unnest(event_params) where event_name = 'scroll' and key = 'percent_scrolled') = 90
        then (select value.string_value from unnest(event_params) where event_name = 'scroll' and key = 'page_location') else null end as scroll_page_90_percent,
    countif(event_name = 'scroll') as scrolls
from 
    `ga.events_20210601`
group by 
    scroll_page_90_percent
order by 
    scrolls desc

Count Outbound Clicks

In [ ]:
%%bigquery --project 

select
    (select value.string_value from unnest(event_params) where event_name = 'click' and key = 'page_location') as page,
    (select value.string_value from unnest(event_params) where event_name = 'click' and key = 'link_domain') as link_domain,
    (select value.string_value from unnest(event_params) where event_name = 'click' and key = 'link_url') as link_url,
    countif(event_name = 'click' and (select value.string_value from unnest(event_params) where event_name = 'click' and key = 'outbound') = 'true') as clicks
from 
    `ga.events_20210601`
group by 
    page,
    link_domain,
    link_url
order by 
    clicks desc

Track Site Search

In [ ]:
%%bigquery --project 

select
    (select value.string_value from unnest(event_params) where event_name = 'view_search_results' and key = 'search_term') as search_term,
    countif(event_name = 'view_search_results') as searches
from 
    `ga.events_20210601`
group by 
    search_term
order by 
    searches desc

Explore Video Engagement

In [ ]:
%%bigquery --project 

select 
    (select value.string_value from unnest(event_params) where event_name like 'video%' and key = 'video_provider') as video_provider,
    (select value.string_value from unnest(event_params) where event_name like 'video%' and key = 'video_title') as video_title,
    (select value.string_value from unnest(event_params) where event_name like 'video%' and key = 'video_url') as video_url,
    (select value.int_value from unnest(event_params) where event_name like 'video%' and key = 'video_duration') as video_duration,
    countif(event_name = 'video_start') as video_start,
    countif(event_name = 'video_progress' and (select value.int_value from unnest(event_params) where event_name = 'video_progress' and key = 'video_percent') = 50) as video_progress_50_percent,
    countif(event_name = 'video_complete') as video_complete
from 
    `ga.events_20210601`
group by 
    video_provider,
    video_title,
    video_url,
    video_duration
order by 
    video_start desc

Count File Downloads

In [ ]:
%%bigquery --project

select
    (select value.string_value from unnest(event_params) where event_name = 'file_download' and key = 'file_extension') as file_type,
    (select value.string_value from unnest(event_params) where event_name = 'file_download' and key = 'file_name') as file_name,
    (select value.string_value from unnest(event_params) where event_name = 'file_download' and key = 'link_text') as link_text,
    (select value.string_value from unnest(event_params) where event_name = 'file_download' and key = 'link_url') as link_url,
    countif(event_name = 'file_download') as downloads
from
    `ga.events_202106*`
group by
    file_type,
    file_name,
    link_text,
    link_url
order by
    downloads desc

Count Events Per User With A Pivot Table

In [ ]:
%%bigquery --project

select
    *
from (
    select
        user_pseudo_id,
        event_name
    from
        `ga.events_202106*`)
pivot (
    count(*)
    for
        event_name
    in (
        'session_start', 
        'first_visit',
        'page_view',
        'scroll',
        'click',
        'view_search_results',
        'file_download', 
        'video_start'))